## 1. 크롬 창 열기 / 클루 로그인

In [3]:
###import BeautifulSoup, import webdriver
###chrome driver, url, page_source
###login


from bs4 import BeautifulSoup
import re
from selenium import webdriver
driver = webdriver.Chrome("/Users/choihyoung-in/Desktop/NAMZ/chromedriver")
klue_url = "http://klue.kr/index.php"
driver.get(klue_url)
html = driver.page_source

login = driver.find_element_by_id("mb_id").send_keys("id")
login = driver.find_element_by_id("mb_pw").send_keys('pw')
login = driver.find_element_by_xpath("""//*[@id="content"]/div[1]/div[2]/div[3]/button[1]""").click()

## 2. 파일 생성 / 이어쓰기

In [4]:
### "w"(write), "a"(add)


lec_info = open("1_info.txt", "w")
lec_reviews = open("1_reviews.txt", "w")
lec_ratings = open("1_ratings.txt", "w")

## 3. 크롤링

In [5]:
###for loop, try-except


for lec_num in range(30000, 45001):
    try:
        lec_url = "http://klue.kr/lecture.php?no={}".format(lec_num)
        driver.get(lec_url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        

        
        
        ################# lec_info ##################        
        
        lec_name = soup.find('div', class_="lec_name").get_text()
        prof_name = soup.find('div', class_="lec_profName").get_text()
        
        ###info_list = semester, code, credit, classification
        info_list = soup.find('div', class_="lec_info").find_all(class_="content")[:4]
        a = []
        for i in info_list:
            info_list = i.get_text()
            a.append(info_list)
        info_list = " | ".join(a)


        lec_time = soup.find_all('span', class_="content")[4].get_text()[:-1]
        lec_time = "".join(lec_time.split("|"))
        
        
        ###e_list = total, difficulty, studytime, attendance, grade, achievement
        b = []
        for i in range(0, 6):
            l = soup.find('div', class_="lec_rating").find_all(class_="left")[i].get_text()
            r = soup.find('div', class_="lec_rating").find_all(class_="right")[i].get_text()
            e_list = l + ": " + r
            b.append(e_list)
        e_list = " | ".join(b)
        
        
        
        
        lec_info_total = " | ".join([str(lec_num), lec_name, prof_name, info_list, lec_time, e_list]) + "\n"
#        print(lec_info_total)
        lec_info.write(lec_info_total)

        

        

        ################ lec_reviews ################        
        
        eval_boxes = soup.find_all('div', class_="content")[:-1]
        eval_num = 0
    
        for eval_box in eval_boxes:
            eval_num += 1
            nickname = eval_box.find('a', class_="bold").get_text()
            ID = eval_box.find('div', class_="wrInfoSub").get_text()[1:5]
    
            lec_eval = eval_box.find('div', class_="wrContent").get_text()
            lec_eval = [word for word in lec_eval.split('\n') if len(word)>0]
            lec_eval = ' '.join(lec_eval)
            lec_eval = re.sub(r" +", " ", lec_eval)
            lec_reviews_total = " | ".join([str(lec_num), str(eval_num), nickname, ID, lec_eval]) + "\n"
#            print(lec_reviews_total)
            lec_reviews.write(lec_reviews_total)





        ################ lec_ratings ################

        eval_boxes = soup.find_all('div', class_="content")[:-1]
        eval_num = 0

        for eval_box in eval_boxes:
            eval_num += 1
            nickname = eval_box.find('a', class_="bold").get_text()
            ID = eval_box.find('div', class_="wrInfoSub").get_text()[1:5]
            a = []
            for i in range(0, 6):
                try:
                    ie_l = eval_box.find('div', class_="right").find_all('div', class_="left")[i]
                    ie_r = eval_box.find('div', class_="right").find_all('div', class_="center")[i]
                    ie = ie_l.get_text() + ": " + str(len(ie_r.find_all(class_="active")))
                    a.append(ie)
                    ie_list = " | ".join(a)
                except:
                    pass        
            lec_ratings_total = " | ".join([str(lec_num), str(eval_num), nickname, ID, ie_list]) + "\n"
#            print(lec_ratings_total)
            lec_ratings.write(lec_ratings_total)

  
    except :
        print("{}번째 강의는 존재하지 않습니다.".format(lec_num))
        pass

## 4. 크롬 창, 파일 닫기

In [6]:
lec_info.close()       
lec_reviews.close()
lec_ratings.close() 
#driver.close()